## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


In [1]:
from google.colab import files

uploaded = files.upload()


Saving chinook.db to chinook.db


In [2]:
from sqlalchemy import create_engine, MetaData

# Conectando com o banco chinook.db
engine = create_engine('sqlite:///chinook.db')


In [3]:
metadata = MetaData()
metadata.reflect(bind=engine)  # Lê todas as tabelas do banco


In [5]:
# conferindo o nome das tabelas
print(metadata.tables.keys())

dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])


In [18]:
from sqlalchemy import select

artist_table = metadata.tables['artists']
# Consulta: selecionar todos os artistas
stmt = select(artist_table)
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
        print(row)


(1, 'AC/DC')
(2, 'Accept')
(3, 'Aerosmith')
(4, 'Alanis Morissette')
(5, 'Alice In Chains')
(6, 'Antônio Carlos Jobim')
(7, 'Apocalyptica')
(8, 'Audioslave')
(9, 'BackBeat')
(10, 'Billy Cobham')
(11, 'Black Label Society')
(12, 'Black Sabbath')
(13, 'Body Count')
(14, 'Bruce Dickinson')
(15, 'Buddy Guy')
(16, 'Caetano Veloso')
(17, 'Chico Buarque')
(18, 'Chico Science & Nação Zumbi')
(19, 'Cidade Negra')
(20, 'Cláudio Zoli')
(21, 'Various Artists')
(22, 'Led Zeppelin')
(23, 'Frank Zappa & Captain Beefheart')
(24, 'Marcos Valle')
(25, 'Milton Nascimento & Bebeto')
(26, 'Azymuth')
(27, 'Gilberto Gil')
(28, 'João Gilberto')
(29, 'Bebel Gilberto')
(30, 'Jorge Vercilo')
(31, 'Baby Consuelo')
(32, 'Ney Matogrosso')
(33, 'Luiz Melodia')
(34, 'Nando Reis')
(35, 'Pedro Luís & A Parede')
(36, 'O Rappa')
(37, 'Ed Motta')
(38, 'Banda Black Rio')
(39, 'Fernanda Porto')
(40, 'Os Cariocas')
(41, 'Elis Regina')
(42, 'Milton Nascimento')
(43, 'A Cor Do Som')
(44, 'Kid Abelha')
(45, 'Sandra De Sá')
(46,

#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [19]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [20]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3.
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [21]:
### Imprima os três primeiros registros da tabela tracks
from sqlalchemy.orm import Session
from sqlalchemy import select

session = Session(engine)
Track = Base.classes.tracks
result = session.execute(select(Track).limit(3)).scalars().all()
for track in result:
    print(track.TrackId, track.Name, track.AlbumId)

1 For Those About To Rock (We Salute You) 1
2 Balls to the Wall 2
3 Fast As a Shark 3


In [22]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
from sqlalchemy import select
from sqlalchemy.orm import Session

session = Session(engine)
Track = Base.classes.tracks
Album = Base.classes.albums

stmt = (
    select(Track.Name, Album.Title)
    .join(Album, Track.AlbumId == Album.AlbumId)
    .limit(20)
)

results = session.execute(stmt).all()
for name, title in results:
    print(f"Faixa: {name} | Álbum: {title}")


Faixa: For Those About To Rock (We Salute You) | Álbum: For Those About To Rock We Salute You
Faixa: Put The Finger On You | Álbum: For Those About To Rock We Salute You
Faixa: Let's Get It Up | Álbum: For Those About To Rock We Salute You
Faixa: Inject The Venom | Álbum: For Those About To Rock We Salute You
Faixa: Snowballed | Álbum: For Those About To Rock We Salute You
Faixa: Evil Walks | Álbum: For Those About To Rock We Salute You
Faixa: C.O.D. | Álbum: For Those About To Rock We Salute You
Faixa: Breaking The Rules | Álbum: For Those About To Rock We Salute You
Faixa: Night Of The Long Knives | Álbum: For Those About To Rock We Salute You
Faixa: Spellbound | Álbum: For Those About To Rock We Salute You
Faixa: Balls to the Wall | Álbum: Balls to the Wall
Faixa: Fast As a Shark | Álbum: Restless and Wild
Faixa: Restless and Wild | Álbum: Restless and Wild
Faixa: Princess of the Dawn | Álbum: Restless and Wild
Faixa: Go Down | Álbum: Let There Be Rock
Faixa: Dog Eat Dog | Álbum: Le

In [23]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
from sqlalchemy import select
from sqlalchemy.orm import Session

session = Session(engine)
InvoiceItem = Base.classes.invoice_items
Track = Base.classes.tracks

stmt = (
    select(Track.Name, InvoiceItem.Quantity)
    .join(Track, InvoiceItem.TrackId == Track.TrackId)
    .limit(10)
)

results = session.execute(stmt).all()
for name, quantity in results:
    print(f"Faixa: {name} | Quantidade vendida: {quantity}")


Faixa: Balls to the Wall | Quantidade vendida: 1
Faixa: Restless and Wild | Quantidade vendida: 1
Faixa: Put The Finger On You | Quantidade vendida: 1
Faixa: Inject The Venom | Quantidade vendida: 1
Faixa: Evil Walks | Quantidade vendida: 1
Faixa: Breaking The Rules | Quantidade vendida: 1
Faixa: Dog Eat Dog | Quantidade vendida: 1
Faixa: Overdose | Quantidade vendida: 1
Faixa: Love In An Elevator | Quantidade vendida: 1
Faixa: Janie's Got A Gun | Quantidade vendida: 1


In [24]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
from sqlalchemy import select, func, desc
from sqlalchemy.orm import Session

session = Session(engine)
InvoiceItem = Base.classes.invoice_items
Track = Base.classes.tracks

stmt = (
    select(Track.Name, func.sum(InvoiceItem.Quantity).label("total_vendido"))
    .join(Track, InvoiceItem.TrackId == Track.TrackId)
    .group_by(Track.TrackId)
    .order_by(desc("total_vendido"))
    .limit(10)
)

results = session.execute(stmt).all()
for name, total in results:
    print(f"Faixa: {name} | Total vendido: {total}")


Faixa: Balls to the Wall | Total vendido: 2
Faixa: Inject The Venom | Total vendido: 2
Faixa: Snowballed | Total vendido: 2
Faixa: Overdose | Total vendido: 2
Faixa: Deuces Are Wild | Total vendido: 2
Faixa: Not The Doctor | Total vendido: 2
Faixa: Por Causa De Você | Total vendido: 2
Faixa: Welcome Home (Sanitarium) | Total vendido: 2
Faixa: Snowblind | Total vendido: 2
Faixa: Cornucopia | Total vendido: 2


In [25]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists
from sqlalchemy import select, func, desc

InvoiceItem = Base.classes.invoice_items
Track = Base.classes.tracks
Album = Base.classes.albums
Artist = Base.classes.artists

stmt = (
    select(Artist.Name, func.sum(InvoiceItem.Quantity).label("total_vendas"))
    .join(Track, InvoiceItem.TrackId == Track.TrackId)
    .join(Album, Track.AlbumId == Album.AlbumId)
    .join(Artist, Album.ArtistId == Artist.ArtistId)
    .group_by(Artist.ArtistId)
    .order_by(desc("total_vendas"))
    .limit(10)
)

results = session.execute(stmt).all()
for artist_name, total in results:
    print(f"Artista: {artist_name} | Total de vendas: {total}")

Artista: Iron Maiden | Total de vendas: 140
Artista: U2 | Total de vendas: 107
Artista: Metallica | Total de vendas: 91
Artista: Led Zeppelin | Total de vendas: 87
Artista: Os Paralamas Do Sucesso | Total de vendas: 45
Artista: Deep Purple | Total de vendas: 44
Artista: Faith No More | Total de vendas: 42
Artista: Lost | Total de vendas: 41
Artista: Eric Clapton | Total de vendas: 40
Artista: R.E.M. | Total de vendas: 39
